In [8]:
from datetime import datetime, date
# html을 파싱
from bs4 import BeautifulSoup
# http request를 보내고 http response 를 받기 위해 urllib
import urllib.request
# regular expression
import re
import os
import sys
from dateutil.parser import parse
import csv

In [9]:
def CRAWL_DATA():
    URL = 'https://www.soccernews.com/soccer-transfers/'
    DIR_NAME = 'csv/'
    FILE_NAME = ''
    FILE_TYPE = '.csv'
    #league_list = ['english-premier-league-transfers', 'spanish-la-liga-transfers', 
    #              'italian-serie-a-transfers', 'german-bundesliga-transfers', 'rest-of-europe-transfers']
    league_list = ['english-premier-league-transfers']
    
    #for year in range(2018, 2006, -1):
    for year in range(2018, 2017, -1):
        full_year = '-' + str(year) + '-' + str(year + 1)
        print('* crawl data for season ', year, ' ~ ', year + 1)
        YEAR = year
        for league in league_list:
            print('* crawl data for league : ' + league)
            FILE_NAME = league
            url = URL + league + full_year + '/'
            print('* created url : %s' % url)
            make_working_directory(DIR_NAME)
            res = crawl(url)
            res = filter_price(res)
            print_list(res)
            #transfers = make_transfers(res)
            #make_csv(transfers, league)

In [10]:
def make_working_directory(dir_name):
    if not os.path.exists(dir_name):
        print('* create directory ', dir_name)
        os.mkdir(dir_name)

In [11]:
def crawl(url):
    source_from_url = urllib.request.urlopen(url)
    print('* crawl from web ' + url)
    # lxml 방식으로 파싱
    soup = BeautifulSoup(source_from_url, 'lxml', from_encoding='utf-8')
    res = []
    # bs4.element.ResultSet, size = 1
    for found in soup.find_all('table', limit=1):
        tmp = found.find_all(text=True)
        filtered = []
        arr = []
        count = 0
        for i in range(len(tmp)):
            item = tmp[i]
            if item != '\n':
                if item != ' ':
                    if item != 'Jan':
                        filtered.append(item)
        # header 제거
        filtered = list(filtered[5:])
        
        for i in range(len(filtered)):
            if count % 6 == 0:
                arr = []
            item = filtered[i]
            if item != '\n':
                if item != ' ':
                    arr.append(item.lower())
                    count = count + 1
            if (count) % 6 == 0:
                if len(arr) is not 0:
                    res.append(arr)
    # print_list(res)
    return res

In [12]:
def filter_price(res):
    data = []
    print('* filter price')
    for i, item in enumerate(res):
        price = item[5]
        if price == 'undisclosed':
            pass
        elif price == 'Loan':
            pass
        elif price == 'Free':
            pass
        elif price.find('million') != -1:
            price = price.split('million')[0]
            price = float(price) * 1000000
            item[5] = int(price)
        data.append(item)
    return data 

In [13]:
def print_list(l):
    print('%d items in list' % len(l))
    for i, item in enumerate(l):
        print(i+1, ' > ', item)

In [14]:
CRAWL_DATA()

* crawl data for season  2018  ~  2019
* crawl data for league : english-premier-league-transfers
* created url : https://www.soccernews.com/soccer-transfers/english-premier-league-transfers-2018-2019/
* create directory  csv/
* crawl from web https://www.soccernews.com/soccer-transfers/english-premier-league-transfers-2018-2019/
* filter price
144 items in list
1  >  ['jan 31', 'leandro bacuna', 'midfielder', 'reading', 'cardiff', 'undisclosed']
2  >  ['jan 31', 'tudor baluta', 'midfielder', 'fc viitorul', 'brighton', 'undisclosed']
3  >  ['jan 31', 'antonio barreca', 'defender', 'monaco', 'newcastle', 'loan']
4  >  ['jan 31', 'ante palaversa', 'midfielder', 'hajduk split', 'man city', 7000000]
5  >  ['jan 31', 'jonny otto', 'defender', 'atletico madrid', 'wolves', 18000000]
6  >  ['jan 31', 'jan mlakar', 'attacker', 'nk maribor', 'brighton', 'undisclosed']
7  >  ['jan 31', 'miguel almiron', 'midfielder', 'atlanta united', 'newcastle united', 21600000]
8  >  ['jan 31', 'lazar markovic